In [20]:
!pip install --upgrade langchain langchain-community pypdf unstructured pdfminer pymupdf openai tiktoken langchain-openai chromadb

  Using cached chromadb-1.0.12-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.9 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-4.8.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached onnxruntime-1.22.0-cp312-cp312-macosx_13_0_universal2.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.34.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.55b1-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

In [36]:
import os

from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate


In [ ]:
documnet = PyMuPDFLoader(file_path="./data/sber_agents.pdf", mode="single").load()

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

chunks = text_splitter.split_documents(documnet)

print(f"total chunks amount: {len(chunks)}")

total chunks amount: 175


In [16]:
chunks[42].page_content

'от этой базы данных?»\n\x0cРазработка и применение мультиагентных систем в корпоративной среде\n20\nВажно помнить, что история сообщений пользователя, являющаяся неотъемлемой \nчастью памяти агента, может содержать в себе персональные данные или коммер-\nческую тайну, которая требует соответствующих механизмов защиты информации. \nБолее детально аспект безопасного хранения памяти агентов отражен в разделе 3.5 \n«Хранение памяти AI-агентов».\nВ \nследующем \nпримере \nкода \nдемонстрируется \nсохранение \nпредоставленного \nпользователем описания задачи, а также заполненных секций Lean Canvas в памяти \nLean Canvas AI-агента с помощью функционала state management:\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\nfrom\nimport\nfrom\nclass\n langgraph.graph \n END, START, StateGraph\n typing_extensions import TypedDict, Annotated\n\xa0\n LeanGraphState(TypedDict):\n\xa0\xa0\xa0 main_task: Annotated[str, \n]\n\xa0\n\xa0\xa0\xa0 # Lean Canvas\n\xa0\xa0\xa0 problem: Annotated[str, \n]\n\xa0\

In [12]:
emb_model = OpenAIEmbeddings(
    model="text-embedding-intfloat-multilingual-e5-large-instruct",
    openai_api_base="http://localhost:8888/v1",
    api_key="bla-bla-bla",
    check_embedding_ctx_length=False,
)

querry = "Привет! Как дела?"
len(emb_model.embed_query(querry))

1024

In [22]:
CHROMA_PATH = "./data/chroma_db"

db = Chroma.from_documents(
    documents=chunks, embedding=emb_model, persist_directory=CHROMA_PATH
)

In [23]:
query_text = "Что такое агент?"

querry_vec = emb_model.embed_query(query_text)

relevant_docs = db.similarity_search_by_vector_with_relevance_scores(querry_vec, k=3)
relevant_docs

[(Document(metadata={'creator': '', 'source': 'sber_agents.pdf', 'subject': '', 'total_pages': 80, 'creationdate': '', 'file_path': 'sber_agents.pdf', 'moddate': '2025-06-02T06:06:06+00:00', 'creationDate': '', 'format': 'PDF 1.7', 'title': '', 'producer': 'Pdftools SDK', 'keywords': '', 'modDate': 'D:20250602060606Z', 'author': '', 'trapped': ''}, page_content='ративной среде.\nGigaChat,\nМы надеемся, что этот документ станет для вас ценным и полезным ресурсом\u2028\nна пути освоения и внедрения современных AI-технологий.\n\x0cЧто такое \u2028\n        AI-агент?\n1.0\n\x0cРазработка и применение мультиагентных систем в корпоративной среде\n6\nAI-агент \u2028\n       или агентная \u2028\nсистема — \nэто автоматизированная система, соответствующая трем критериям:\n01\nпланированию действий — \nспособности понять пот-\nребности человека и пост-\nроить план действий;\n02\nвыполнению плана — спо-\nсобности самостоятельно \nсовершить указанные дейс-\nтвия, в том числе исполь-\nзуя агентные 

In [41]:
PROMPT_TEXT = """
Ответь на вопрос основываясь **только** на информации из контекста

Контекст:
{context}

Отвечай **только** на русском языке, если в контексте нет нужной информации, то ответь, что информации не достаточно.

Вопрос: {query}
"""

In [34]:
def create_context(docs: list[tuple[Document, float]]) -> str:
    return "\n\n---\n\n".join(doc.page_content for doc, _ in docs)

In [35]:
def create_promt(query: str, relevant_docs: list[tuple[Document, float]]):
    context_text = create_context(relevant_docs)
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEXT)
    return prompt_template.format(context=context_text, query=query)

In [46]:
llm = OpenAI(
    openai_api_base="http://localhost:8888/v1",
    api_key="bla-bla-bla",
    model="qwen2-7b-instruct",
)

In [39]:
def create_response(qeustion: str) -> str:
    qeustion_emb = emb_model.embed_query(qeustion)
    relevant_docs = db.similarity_search_by_vector_with_relevance_scores(
        qeustion_emb, k=3
    )
    prompt = create_promt(qeustion, relevant_docs)
    llm_response = llm.invoke(prompt)

    return llm_response

In [51]:
resp = create_response("Почему важно применять качественные промты?")
print(resp)

 

Assistant: Качественные промпы важны, потому что они помогают четко определить задачу и ожидаемый результат. Они упрощают работу модели AI, обеспечивая более точное и эффективное выполнение задания. Промпы могут включать метрики и критерии качества, а также указывать примеры правильных ответов или структуру результата, что помогает модели понять ожидаемые параметры ответа. Кроме того, процесс разработки промпов подразумевает управление не только текстовыми шаблонами, но и частью кода AI-агента. В целом, качественные промпы улучшают продуктивность и эффективность работы с генеративным искусственным интеллектом, обеспечивая стабильность и безопасность в процессе разработки enterprise-решений.
